# Analyse

In [1]:
import ast
import json

In [2]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import sqlite3
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from pandas_profiling import ProfileReport

## Laden aus DWH

In [3]:
SQLITE_FILE_PATH = os.path.join('..', "output", "dwh.sqlite3")
PROFILE_PATH = os.path.join('..','output', 'movies_report.html')

In [4]:
SQLITE_FILE_PATH

'..\\output\\dwh.sqlite3'

In [5]:
connection = sqlite3.connect(SQLITE_FILE_PATH)

In [6]:
genres = pd.read_sql_query("SELECT * from genres;", connection)
genres

,genre_id,genre_name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
5,10749,Romance
6,18,Drama
7,28,Action
8,80,Crime
9,53,Thriller


In [7]:
movies = pd.read_sql_query("SELECT * from movies;", connection)
movies.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,None,Toy Story,0.0,7.7,5415.0
1,False,65000000,None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,0.0,6.9,2413.0
2,False,0,None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,0.0,6.5,92.0
3,False,16000000,None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,0.0,6.1,34.0
4,False,0,None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,0.0,5.7,173.0


In [8]:
movies_genre = pd.read_sql_query("SELECT * from movies_genre;", connection)
movies_genre.head()

,movie_id,genre_id
0,862,16
1,862,35
2,862,10751
3,8844,12
4,8844,14


## Erster Überblick

In [9]:
profile = movies.profile_report(title="Pandas Profiling Report", minimal=True)
profile.to_file(PROFILE_PATH)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Spalten in SQL - Durchschnittliche Bewertung pro Genre

In [10]:
query = """
SELECT AVG(movies.vote_average) AS vote_avg, genres.genre_name FROM genres
JOIN movies_genre ON genres.genre_id = movies_genre.genre_id
JOIN movies ON movies.id = movies_genre.movie_id
GROUP BY genres.genre_name
ORDER BY vote_avg DESC
"""

avg_votes_df = pd.read_sql_query(query, con=connection)
avg_votes_df

,vote_avg,genre_name
0,6.274459,Animation
1,6.154220,History
2,6.041358,War
3,5.903890,Drama
4,5.879921,Crime
5,5.878500,Music
6,5.823222,Documentary
7,5.823071,Mystery
8,5.800815,Romance
9,5.781646,Fantasy
